In [1]:
%matplotlib 
%load_ext autoreload
%autoreload 2

Using matplotlib backend: Qt4Agg


In [2]:
import sys
sys.path.append("/mnt/Data/Workspaces/Eclipse/dataAnalysis/Sleep-current/src/")

In [3]:
import signale
import numpy as np
import matplotlib.pyplot as pl
import spectrum as sp
import pickle as pkl
import scipy.signal as scsig
import pywt
import colormaps as mycmps
import signale.tools as tools
import os
import itertools
import pandas as pd
import statsmodels.api as sm
import seaborn as sns

/home/chenani/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)
/home/chenani/anaconda/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [4]:
animalPath = '/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3656/LinearTrack_03-05-2015_Day3/03-sleep2/'

In [5]:
lfpPaths = []
for lfp in tools.locate('*.lfp',animalPath):
    lfpPaths.append(os.path.join(lfp[0],lfp[1]))
lfpPaths = sorted(lfpPaths)
[ path for path in lfpPaths if (path.find('leep')>0 and path.find('CS')>0) ]


['/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3656/LinearTrack_03-05-2015_Day3/03-sleep2/CSC1.lfp',
 '/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3656/LinearTrack_03-05-2015_Day3/03-sleep2/CSC10.lfp',
 '/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3656/LinearTrack_03-05-2015_Day3/03-sleep2/CSC11-post.lfp',
 '/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3656/LinearTrack_03-05-2015_Day3/03-sleep2/CSC11.lfp',
 '/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3656/LinearTrack_03-05-2015_Day3/03-sleep2/CSC12.lfp',
 '/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3656/LinearTrack_03-05-2015_Day3/03-sleep2/CSC13.lfp',
 '/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3656/LinearTrack_03-05-2015_Day3/03-sleep2/CSC14.lfp',
 '/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3656/LinearTrack_03-05-2015_Day3/03-sleep2/CSC15.lfp',
 '/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3656/LinearTrack_03-05-2015_Day3/0

In [6]:
lfps =[]
for item in lfpPaths:
    try:
        lfps.append(pkl.load(open(item,'rb')))
    except ValueError:
        print  "%s is broken or contains no data!" %(item.split('/')[-1])

In [7]:
Nlfp = len(lfpPaths[:])
f,ax = pl.subplots((Nlfp/4)+1,4,sharex=True,sharey=False)
for pp, path in enumerate(lfpPaths[:]):
    ax[pp/4,np.mod(pp,4)].axvspan(150,225,alpha = 0.5,color='g')
    print path
    lfp = lfps[pp]
    dsig = scsig.decimate(lfp.signal_white,2)
    p = sp.Periodogram(dsig,sampling=1000,NFFT=2**14)
    p()
    ax[pp/4,np.mod(pp,4)].plot(p.frequencies(),40*p.psd,alpha=0.75,)
    ax[pp/4,np.mod(pp,4)].plot(p.frequencies(),np.convolve(p.psd,scsig.gaussian(p.psd.size/50,p.psd.size/100),'same'),c='m')
    for fff in range(0,300,50):
        ax[pp/4,np.mod(pp,4)].axvline(fff,c='r')
    ax[pp/4,np.mod(pp,4)].legend([lfp.tags['file'].split('/')[-1],'6',lfp.tags['channel']])
    ax[pp/4,np.mod(pp,4)].set_xlim(0,300)
f,ax = pl.subplots((Nlfp/4)+1,4,sharex=True,sharey=False)
for pp, path in enumerate(lfpPaths[:]):
    print path
    lfp = lfps[pp]
    dsig = scsig.decimate(lfp.signal,2)
    ax[pp/4,np.mod(pp,4)].plot(dsig,lw=1,label =lfp.tags['file'].split('/')[-1])
    ax[pp/4,np.mod(pp,4)].legend() 

/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3656/LinearTrack_03-05-2015_Day3/03-sleep2/CSC1.lfp
/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3656/LinearTrack_03-05-2015_Day3/03-sleep2/CSC10.lfp
/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3656/LinearTrack_03-05-2015_Day3/03-sleep2/CSC11-post.lfp
/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3656/LinearTrack_03-05-2015_Day3/03-sleep2/CSC11.lfp
/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3656/LinearTrack_03-05-2015_Day3/03-sleep2/CSC12.lfp
/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3656/LinearTrack_03-05-2015_Day3/03-sleep2/CSC13.lfp
/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3656/LinearTrack_03-05-2015_Day3/03-sleep2/CSC14.lfp
/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3656/LinearTrack_03-05-2015_Day3/03-sleep2/CSC15.lfp
/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3656/LinearTrack_03-05-2015_Day3/03-sleep2/CSC16.lfp
/home/chenani/d

/home/chenani/anaconda/lib/python2.7/site-packages/spectrum-0.6.1-py2.7-linux-x86_64.egg/spectrum/psd.py:451: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if self.__psd == None:


In [25]:
nn=0
pkl.dump(lfps[nn],open(lfps[nn].tags['file'].split('.')[0]+'-track.lfp','wb'),pkl.HIGHEST_PROTOCOL)

IOError: [Errno 2] No such file or directory: '/home/ephysdata/dataWork/ali/MECLesion_SleepData/Marta/Rat3656/LinearTrack_03-04-2015_Day2/02-2015-03-04_13-51-58/CSC1-track.lfp'